In [14]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:

driver = webdriver.Chrome()
driver.get('https://www.yvr.ca/en/passengers/flights/arriving-flights')

html = driver.page_source
soup = BeautifulSoup(html)

for tag in soup.find_all("a", class_="my_class"):
    print(tag.text)

In [7]:
# Direct tag/class fileters for flight numbers, airline and origin
flight_nums = soup.findAll("td", {"class": "yvr-table__cell yvr-flights__flightNumber notranslate"})
airline = soup.findAll("td", {"class": "hidden-until--md yvr-table__cell yvr-table__cell--fade-out yvr-table__cell--nowrap notranslate"})
origins = soup.findAll("td", {"class": "yvr-table__cell yvr-table__cell--fade-out yvr-table__cell--nowrap notranslate"})

# Collect row data
raw_rows = soup.tbody.findAll('tr', {'class':'yvr-flights__row'})
raw_rows.pop(0)

#Each row contains at least the scheduled arrival time or a pair of times
pairs = list(map(lambda x: x.findAll('time'), raw_rows))
sched = []
est = []
for i in pairs:
    if len(i) == 2:
        sched.append(i[0]['datetime'])
        est.append(i[1]['datetime'])
        
    if len(i) == 1:
        sched.append(i[0]['datetime'])
        est.append(np.nan)

flight_ids = list(map(lambda x: x['id'],raw_rows))
flight_nums = list(map(lambda x: x.text, flight_nums))
airline = list(map(lambda x: x.text, airline))
origins = list(map(lambda x: x.text, origins))

In [8]:
#Create dataframe

YVR_arrivals = pd.DataFrame({"id": flight_ids,
                             "sched": sched,
                             "estimated": est,
                             "flight_number": flight_nums, 
                             "airline": airline, 
                             "origin":origins})

In [9]:
YVR_arrivals.loc[2:4,:]

,id,sched,estimated,flight_number,airline,origin
2,299835271,2020-02-10T10:35:00-08:00,2020-02-10T10:35:00-08:00,AC64,Air Canada,Seoul-Incheon
3,299834153,2020-02-10T10:36:00-08:00,2020-02-10T10:36:00-08:00,AC8060,Air Canada,Victoria
4,299836056,2020-02-10T10:40:00-08:00,2020-02-10T10:40:00-08:00,WS3321,WestJet,Kelowna


In [34]:
date_stamp =str(datetime.date(datetime.now()))
YVR_arrivals.to_csv('../data/arrivals_'+date_stamp+'.csv')